In [38]:
import pandas as pd
from TimetableBot.settings import BASE_DIR
import os
from fuzzywuzzy import process
import re
from typing import Tuple, Dict

In [115]:
def read_timetable(path:str)->pd.DataFrame:
    skiprows = list(range(16))
    print(skiprows)
    try:
        df = pd.read_excel(path, skiprows=lambda ind: ind in skiprows)
        df = df.ffill()
        return df
    except Exception:
        return "Выбран не подходящий файл"

In [116]:
def get_cleaned_group_names(df: pd.DataFrame) -> Tuple:
    return tuple([col for col in df.columns if "Unnamed" not in col])

In [117]:
def get_only_group_numbers(df: pd.DataFrame, cleaned_group_names: Tuple|None = None) -> Tuple:
    if cleaned_group_names is None:
        cleaned_group_names = get_cleaned_group_names(df=df)
    pattern = r"\d\d-\d\d\d"
    return tuple([re.search(pattern, group).group() for group in cleaned_group_names])

def get_groups_dict_by_group_numbers(df: pd.DataFrame, cleaned_group_names: Tuple|None = None) -> Dict:
    if cleaned_group_names is None:
        cleaned_group_names = get_cleaned_group_names(df=df)
    pattern = r"\d\d-\d\d\d"
    group_dict = {}
    for group in cleaned_group_names:
        g_data = re.search(pattern, group)
        group_num, group_full_name = g_data.group(), g_data.string 
        if group_dict.get(group_num):
            group_dict[group_num].append(group_full_name)
            continue
        group_dict.update({
            group_num:[group_full_name]
        })
    return group_dict

In [118]:
def group_num_validator(user_input: str) -> bool:
    pattern = r"\d\d-\d\d\d"
    return True if re.search(pattern, user_input) else False
    
def get_group_num(user_input: str) -> bool:
    pattern = r"\d\d-\d\d\d"
    return re.search(pattern, user_input).group()
# print(group_num_validator("qweqweqwe"))
# print(group_num_validator("'09-301': ['09-302 (1)', '09-303 (2)'],"))
# print(group_num_validator("qweq09-301weqwe"))
# print(group_num_validator("09-301"))
# print(group_num_validator(""))


In [119]:
class NotGroupNumStyle(Exception):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return f'NotGroupNumStyle, {self.message} '
        else:
            return 'NotGroupNumStyle has been raised'
            
    def get_message(self):
        return self.message

class GroupNotFound(Exception):
    def __init__(self, *args):
        if args:
            self.message = args[0]
        else:
            self.message = None

    def __str__(self):
        if self.message:
            return f'GroupNotFound, {self.message} '
        else:
            return 'GroupNotFound has been raised'
            
    def get_message(self):
        return self.message

In [120]:
def get_founded_group_names(user_input:str, groups_dict: dict)-> list[str]:
    if group_num_validator(user_input):
        g_num = get_group_num(user_input)
        if result := groups_dict.get(g_num):
            return result
        raise GroupNotFound(f"Введенный номер группы - {g_num}, не найден в рассписании")
    else:
        raise NotGroupNumStyle("Введен неверный формат номера группы.\nНомер должен быть вида XX-XXX,  например 01-001")


In [121]:
path = os.path.join(BASE_DIR, "actual_timetable.xlsx")
tt_df = read_timetable(path=path)
group_names = get_cleaned_group_names(df=tt_df)
only_numbers = get_only_group_numbers(df=tt_df, cleaned_group_names=group_names)
groups_dict = get_groups_dict_by_group_numbers(df=tt_df, cleaned_group_names=group_names)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [122]:
# этот блок и те что ниже - юзлесс 

user_inputs = ["qweqweqwe", "'09-301': ['09-302 (1)', '09-303 (2)'],","qweq09-301weqwe","09-301","", "09-315"]
for u_input in user_inputs:
    try:
        group_names = get_founded_group_names(user_input=u_input, groups_dict=groups_dict)
        print(group_names)
    except (GroupNotFound, NotGroupNumStyle) as e:
        print(e.get_message())

Введен неверный формат номера группы.
Номер должен быть вида XX-XXX,  например 01-001
['09-301 (1)', '09-301 (2)']
['09-301 (1)', '09-301 (2)']
['09-301 (1)', '09-301 (2)']
Введен неверный формат номера группы.
Номер должен быть вида XX-XXX,  например 01-001
['09-315 (1) (Анализ данных и его приложения)', '09-315 (2) (Математическое моделирование физических процессов) ', '09-315 (3) (Математические методы и информационные технологии в экономике и финансах)', '09-315 (4) (Искусственный интеллект и суперкомпьютерные вычисления)']


In [123]:
selected_group = '09-315 (1) (Анализ данных и его приложения)'

In [124]:
by_group = tt_df[['Unnamed: 0', 'Unnamed: 1']]
df = by_group.rename(columns={'Unnamed: 0': 'day', 'Unnamed: 1': 'time'})

In [150]:
day_dict:[str, list] = {}
for row in df.itertuples():
    index, day = row[:2]
    if len(day)==2:
        continue
    if day_dict.get(day):
        day_dict[day].append(index)
    else:
        day_dict.update({
            day: [index]
        })
    if row[0] == 89:
        break
print(day_dict)
for k,v in day_dict.items():
    print(k, len(v))тут

{'понедельник': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 'вторник': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], 'среда': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43], 'четверг': [45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 'пятница': [61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74], 'суббота': [76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]}
понедельник 14
вторник 14
среда 14
четверг 15
пятница 14
суббота 14


In [ ]:
for row in df.itertuples():
    print(row, end="\n\n\n")

In [ ]:
for data in tt.itertuples():
    print(data[3], data[4], end="\n\n\n", sep="\n_____\n")